In [1]:
data_file = "../data/raw/junkfood_because.tsv"

In [2]:
with open(data_file) as i:
    sentences = [line.strip() for line in i]
    
print(sentences[:3])

['because it is unhealthy and can cause Type 2 diabetes, and that is a terrible way to generate income in a school.\tDiabetes and Risk Factors', 'because it encourages children to continue eating unhealthy snacks and items that can lead to serious health consequences such as obesity and Diabetes.\tDiabetes and Risk Factors', 'because it is unhealthy for children and increases the likelihood of getting Type 2 Diabetes.\tDiabetes and Risk Factors']


In [3]:
import spacy

nlp = spacy.load("en")

I0225 14:47:07.845747 140105628034880 file_utils.py:38] PyTorch version 1.1.0 available.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/li

In [4]:
spacy_docs = list(nlp.pipe(sentences))

In [5]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 3 and not t.is_stop] for doc in spacy_docs]
print(docs[:3])

[['unhealthy', 'cause', 'type', 'diabetes', 'terrible', 'generate', 'income', 'school', 'diabetes', 'risk', 'factor'], ['encourage', 'child', 'continue', 'eat', 'unhealthy', 'snack', 'item', 'lead', 'health', 'consequence', 'obesity', 'diabetes', 'diabetes', 'risk', 'factor'], ['unhealthy', 'child', 'increase', 'likelihood', 'get', 'type', 'diabete', 'diabetes', 'risk', 'factor']]


In [6]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # bigrams can be recognized by the "_" that joins the invidual words
            docs[idx].append(token)

I0225 14:47:09.858697 140105628034880 textcleaner.py:37] 'pattern' package not found; tag filters are not available for English
I0225 14:47:09.861663 140105628034880 phrases.py:475] collecting all words and their counts
I0225 14:47:09.862245 140105628034880 phrases.py:482] PROGRESS: at sentence #0, processed 0 words and 0 word types
I0225 14:47:09.866201 140105628034880 phrases.py:505] collected 1437 word types from a corpus of 3459 words (unigram + bigrams) and 392 sentences
I0225 14:47:09.866521 140105628034880 phrases.py:558] using 1437 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>


In [7]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents:', len(dictionary))

dictionary.filter_extremes(no_below=2, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

print("Example representation of document 3:", dictionary.doc2bow(docs[2]))

I0225 14:47:09.879328 140105628034880 dictionary.py:202] adding document #0 to Dictionary(0 unique tokens: [])
I0225 14:47:09.882991 140105628034880 dictionary.py:209] built Dictionary(356 unique tokens: ['cause', 'diabetes', 'factor', 'generate', 'income']...) from 392 documents (total 3548 corpus positions)
I0225 14:47:09.883807 140105628034880 dictionary.py:373] discarding 214 tokens: [('diabetes', 392), ('factor', 374), ('generate', 1), ('income', 1), ('risk', 375), ('terrible', 1), ('unhealthy', 273), ('consequence', 1), ('item', 1), ('culprit', 1)]...
I0225 14:47:09.884143 140105628034880 dictionary.py:376] keeping 142 tokens which were in no less than 2 and no more than 196 (=50.0%) documents
I0225 14:47:09.884508 140105628034880 dictionary.py:381] resulting dictionary: Dictionary(142 unique tokens: ['cause', 'school', 'type', 'child', 'continue']...)


Number of unique words in original documents: 356
Number of unique words after removing rare and common words: 142
Example representation of document 3: [(2, 1), (3, 1), (11, 1), (12, 1), (13, 1), (14, 1)]


In [8]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [9]:
from gensim.models import LdaModel

%time model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, chunksize=100, passes=20, random_state=1)

I0225 14:47:09.900561 140105628034880 ldamodel.py:557] using symmetric alpha at 0.2
I0225 14:47:09.900962 140105628034880 ldamodel.py:557] using symmetric eta at 0.2
I0225 14:47:09.901464 140105628034880 ldamodel.py:481] using serial LDA version on this node
I0225 14:47:09.902057 140105628034880 ldamodel.py:919] running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 392 documents, updating model once every 100 documents, evaluating perplexity every 392 documents, iterating 50x with a convergence threshold of 0.001000
I0225 14:47:09.902317 140105628034880 ldamodel.py:968] PROGRESS: pass 0, at document #100/392
I0225 14:47:09.929162 140105628034880 ldamodel.py:230] merging changes from 100 documents into a model of 392 documents
I0225 14:47:09.929887 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.128*"high" + 0.097*"type" + 0.065*"cause" + 0.065*"lead" + 0.065*"student" + 0.065*"kid" + 0.034*"diabete" + 0.034*"rate" + 0.033*"year" + 0.033*"recent

I0225 14:47:10.010030 140105628034880 ldamodel.py:1037] topic diff=0.500997, rho=0.410997
I0225 14:47:10.010299 140105628034880 ldamodel.py:968] PROGRESS: pass 1, at document #200/392
I0225 14:47:10.022401 140105628034880 ldamodel.py:230] merging changes from 100 documents into a model of 392 documents
I0225 14:47:10.022840 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.278*"student" + 0.101*"kid" + 0.059*"lifestyle" + 0.037*"lead" + 0.032*"choose" + 0.027*"life" + 0.024*"teach" + 0.024*"easy" + 0.020*"country" + 0.017*"term"
I0225 14:47:10.023179 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.177*"obesity" + 0.093*"promote" + 0.061*"habit" + 0.060*"type" + 0.047*"increase" + 0.040*"diabete" + 0.040*"eating" + 0.039*"young" + 0.038*"rate" + 0.030*"child"
I0225 14:47:10.023491 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.162*"health" + 0.086*"child" + 0.068*"school" + 0.058*"choice" + 0.054*"good" + 0.052*"cause" + 0.043*"problem" + 0.040*"encourage" + 0.034*"poor

I0225 14:47:10.098211 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.140*"obesity" + 0.116*"promote" + 0.085*"habit" + 0.060*"eating" + 0.053*"type" + 0.039*"young" + 0.038*"increase" + 0.038*"child" + 0.035*"diabete" + 0.031*"eat"
I0225 14:47:10.098571 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.225*"health" + 0.084*"school" + 0.072*"child" + 0.070*"choice" + 0.069*"good" + 0.038*"encourage" + 0.036*"cause" + 0.031*"problem" + 0.026*"issue" + 0.022*"negative"
I0225 14:47:10.098887 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.144*"food" + 0.107*"junk" + 0.103*"junk_food" + 0.082*"nutritional" + 0.082*"value" + 0.051*"nutritional_value" + 0.040*"kid" + 0.032*"little" + 0.028*"little_nutritional" + 0.020*"learn"
I0225 14:47:10.099188 140105628034880 ldamodel.py:1159] topic #4 (0.200): 0.307*"healthy" + 0.063*"child" + 0.056*"option" + 0.042*"food" + 0.040*"need" + 0.040*"available" + 0.031*"lifestyle" + 0.024*"help" + 0.019*"contribute" + 0.019*"responsibility"


I0225 14:47:10.177164 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.278*"health" + 0.078*"good" + 0.071*"school" + 0.071*"child" + 0.056*"choice" + 0.039*"cause" + 0.033*"problem" + 0.030*"effect" + 0.030*"encourage" + 0.027*"issue"
I0225 14:47:10.177465 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.167*"food" + 0.138*"junk" + 0.130*"junk_food" + 0.068*"value" + 0.064*"nutritional" + 0.041*"kid" + 0.040*"nutritional_value" + 0.025*"little" + 0.022*"little_nutritional" + 0.019*"learn"
I0225 14:47:10.177767 140105628034880 ldamodel.py:1159] topic #4 (0.200): 0.358*"healthy" + 0.078*"child" + 0.068*"option" + 0.044*"responsibility" + 0.043*"need" + 0.033*"food" + 0.032*"available" + 0.026*"help" + 0.017*"affect" + 0.016*"lifestyle"
I0225 14:47:10.178115 140105628034880 ldamodel.py:1037] topic diff=0.221221, rho=0.355335
I0225 14:47:10.178385 140105628034880 ldamodel.py:968] PROGRESS: pass 4, at document #100/392
I0225 14:47:10.187791 140105628034880 ldamodel.py:230] mergin

I0225 14:47:10.244180 140105628034880 ldamodel.py:1159] topic #4 (0.200): 0.333*"healthy" + 0.083*"child" + 0.060*"option" + 0.047*"need" + 0.042*"responsibility" + 0.034*"food" + 0.032*"help" + 0.029*"available" + 0.015*"affect" + 0.015*"prove"
I0225 14:47:10.244457 140105628034880 ldamodel.py:1037] topic diff=0.273073, rho=0.317500
I0225 14:47:10.244728 140105628034880 ldamodel.py:968] PROGRESS: pass 5, at document #200/392
I0225 14:47:10.253751 140105628034880 ldamodel.py:230] merging changes from 100 documents into a model of 392 documents
I0225 14:47:10.254184 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.334*"student" + 0.101*"kid" + 0.080*"lifestyle" + 0.034*"choose" + 0.029*"lead" + 0.029*"life" + 0.026*"teach" + 0.023*"easy" + 0.018*"country" + 0.018*"long"
I0225 14:47:10.254522 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.175*"obesity" + 0.086*"promote" + 0.075*"type" + 0.054*"habit" + 0.051*"increase" + 0.050*"diabete" + 0.045*"contribute" + 0.038*"child" + 

...

I0225 14:47:10.791573 140105628034880 ldamodel.py:1037] topic diff=0.133144, rho=0.224055
I0225 14:47:10.803575 140105628034880 ldamodel.py:814] -4.496 per-word bound, 22.6 perplexity estimate based on a held-out corpus of 92 documents with 381 words
I0225 14:47:10.803808 140105628034880 ldamodel.py:968] PROGRESS: pass 15, at document #392/392
I0225 14:47:10.810776 140105628034880 ldamodel.py:230] merging changes from 92 documents into a model of 392 documents
I0225 14:47:10.811293 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.335*"student" + 0.105*"kid" + 0.077*"lifestyle" + 0.040*"life" + 0.039*"choose" + 0.036*"teach" + 0.026*"lead" + 0.023*"long" + 0.023*"term" + 0.021*"harmful"
I0225 14:47:10.811631 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.135*"obesity" + 0.115*"promote" + 0.083*"habit" + 0.060*"type" + 0.048*"eating" + 0.044*"increase" + 0.041*"child" + 0.040*"diabete" + 0.038*"young" + 0.037*"contribute"
I0225 14:47:10.811947 140105628034880 ldamodel.py:1159

I0225 14:47:10.873259 140105628034880 ldamodel.py:230] merging changes from 100 documents into a model of 392 documents
I0225 14:47:10.873841 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.356*"student" + 0.109*"kid" + 0.070*"lifestyle" + 0.036*"life" + 0.035*"choose" + 0.033*"teach" + 0.024*"lead" + 0.021*"long" + 0.021*"term" + 0.019*"harmful"
I0225 14:47:10.874158 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.130*"obesity" + 0.093*"type" + 0.077*"promote" + 0.062*"diabete" + 0.054*"increase" + 0.050*"habit" + 0.041*"lead" + 0.041*"contribute" + 0.037*"child" + 0.034*"high"
I0225 14:47:10.874468 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.271*"health" + 0.075*"school" + 0.072*"good" + 0.059*"child" + 0.059*"choice" + 0.046*"encourage" + 0.039*"problem" + 0.033*"cause" + 0.027*"effect" + 0.025*"issue"
I0225 14:47:10.874783 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.142*"food" + 0.118*"junk" + 0.111*"junk_food" + 0.086*"nutritional" + 0.085*"value" +

I0225 14:47:10.936589 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.160*"obesity" + 0.085*"promote" + 0.076*"type" + 0.055*"habit" + 0.050*"diabete" + 0.050*"increase" + 0.043*"contribute" + 0.041*"child" + 0.038*"lead" + 0.035*"eating"
I0225 14:47:10.936888 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.249*"health" + 0.078*"school" + 0.065*"good" + 0.061*"choice" + 0.057*"child" + 0.055*"encourage" + 0.044*"problem" + 0.031*"cause" + 0.026*"effect" + 0.022*"issue"
I0225 14:47:10.937205 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.133*"food" + 0.105*"nutritional" + 0.105*"value" + 0.104*"junk" + 0.097*"junk_food" + 0.064*"nutritional_value" + 0.038*"little" + 0.037*"kid" + 0.037*"little_nutritional" + 0.015*"get"
I0225 14:47:10.937506 140105628034880 ldamodel.py:1159] topic #4 (0.200): 0.299*"healthy" + 0.089*"child" + 0.052*"option" + 0.046*"need" + 0.041*"responsibility" + 0.040*"available" + 0.036*"food" + 0.027*"help" + 0.022*"make" + 0.020*"decision"
I0225

I0225 14:47:11.000010 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.148*"food" + 0.112*"junk" + 0.106*"junk_food" + 0.090*"nutritional" + 0.090*"value" + 0.055*"nutritional_value" + 0.041*"kid" + 0.034*"little" + 0.031*"little_nutritional" + 0.018*"learn"
I0225 14:47:11.000306 140105628034880 ldamodel.py:1159] topic #4 (0.200): 0.323*"healthy" + 0.084*"child" + 0.059*"option" + 0.044*"need" + 0.042*"available" + 0.037*"responsibility" + 0.035*"food" + 0.027*"help" + 0.022*"decision" + 0.017*"behavior"
I0225 14:47:11.000655 140105628034880 ldamodel.py:1037] topic diff=0.118487, rho=0.204465
I0225 14:47:11.012212 140105628034880 ldamodel.py:814] -4.493 per-word bound, 22.5 perplexity estimate based on a held-out corpus of 92 documents with 381 words
I0225 14:47:11.012485 140105628034880 ldamodel.py:968] PROGRESS: pass 19, at document #392/392
I0225 14:47:11.019069 140105628034880 ldamodel.py:230] merging changes from 92 documents into a model of 392 documents
I0225 14:47:11.01964

CPU times: user 1.15 s, sys: 159 ms, total: 1.31 s
Wall time: 1.12 s


In [10]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words)

I0225 14:47:11.024559 140105628034880 ldamodel.py:1159] topic #0 (0.200): 0.336*"student" + 0.106*"kid" + 0.078*"lifestyle" + 0.040*"life" + 0.039*"choose" + 0.036*"teach" + 0.026*"lead" + 0.023*"long" + 0.023*"term" + 0.021*"harmful"
I0225 14:47:11.025077 140105628034880 ldamodel.py:1159] topic #1 (0.200): 0.136*"obesity" + 0.112*"promote" + 0.081*"habit" + 0.062*"type" + 0.047*"eating" + 0.044*"increase" + 0.041*"child" + 0.041*"diabete" + 0.038*"contribute" + 0.037*"lead"
I0225 14:47:11.025439 140105628034880 ldamodel.py:1159] topic #2 (0.200): 0.276*"health" + 0.077*"good" + 0.076*"school" + 0.060*"choice" + 0.056*"child" + 0.046*"encourage" + 0.036*"problem" + 0.031*"cause" + 0.030*"effect" + 0.027*"issue"
I0225 14:47:11.026001 140105628034880 ldamodel.py:1159] topic #3 (0.200): 0.158*"food" + 0.126*"junk" + 0.119*"junk_food" + 0.080*"value" + 0.078*"nutritional" + 0.047*"nutritional_value" + 0.042*"kid" + 0.030*"little" + 0.027*"little_nutritional" + 0.018*"learn"
I0225 14:47:11.

1 : 0.336*"student" + 0.106*"kid" + 0.078*"lifestyle" + 0.040*"life" + 0.039*"choose" + 0.036*"teach" + 0.026*"lead" + 0.023*"long" + 0.023*"term" + 0.021*"harmful"
2 : 0.136*"obesity" + 0.112*"promote" + 0.081*"habit" + 0.062*"type" + 0.047*"eating" + 0.044*"increase" + 0.041*"child" + 0.041*"diabete" + 0.038*"contribute" + 0.037*"lead"
3 : 0.276*"health" + 0.077*"good" + 0.076*"school" + 0.060*"choice" + 0.056*"child" + 0.046*"encourage" + 0.036*"problem" + 0.031*"cause" + 0.030*"effect" + 0.027*"issue"
4 : 0.158*"food" + 0.126*"junk" + 0.119*"junk_food" + 0.080*"value" + 0.078*"nutritional" + 0.047*"nutritional_value" + 0.042*"kid" + 0.030*"little" + 0.027*"little_nutritional" + 0.018*"learn"
5 : 0.345*"healthy" + 0.091*"child" + 0.064*"option" + 0.051*"responsibility" + 0.043*"need" + 0.035*"available" + 0.032*"food" + 0.027*"help" + 0.018*"decision" + 0.016*"affect"


In [11]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.179588 -0.003304       1        1  12.078008
1     -0.064585 -0.314885       2        1  27.379200
2     -0.274085  0.050256       3        1  19.493086
3      0.241233  0.049607       4        1  25.180038
4     -0.082151  0.218326       5        1  15.869667, topic_info=     Category       Freq               Term      Total  loglift  logprob
term                                                                    
65    Default  96.000000            healthy  96.000000  30.0000  30.0000
7     Default  94.000000             health  94.000000  29.0000  29.0000
52    Default  71.000000            student  71.000000  28.0000  28.0000
9     Default  65.000000            obesity  65.000000  27.0000  27.0000
16    Default  78.000000               food  78.000000  26.0000  26.0000
18    Default  56.000000               junk  56.000000  25.0000  25.0000
19    Default  53.000000          junk_food  53.000000  24.0000  24.0000
23    Default  54.000000            promote  54.000000  23.0000  23.0000
37    Default  41.000000                kid  41.000000  22.0000  22.0000
36    Default  39.000000              habit  39.000000  21.0000  21.0000
53    Default  35.000000              value  35.000000  20.0000  20.0000
51    Default  34.000000        nutritional  34.000000  19.0000  19.0000
74    Default  26.000000               good  26.000000  18.0000  18.0000
1     Default  26.000000             school  26.000000  17.0000  17.0000
3     Default  64.000000              child  64.000000  16.0000  16.0000
91    Default  17.000000          lifestyle  17.000000  15.0000  15.0000
2     Default  30.000000               type  30.000000  14.0000  14.0000
129   Default  18.000000             option  18.000000  13.0000  13.0000
30    Default  21.000000             choice  21.000000  12.0000  12.0000
67    Default  21.000000  nutritional_value  21.000000  11.0000  11.0000
35    Default  23.000000             eating  23.000000  10.0000  10.0000
13    Default  21.000000           increase  21.000000   9.0000   9.0000
6     Default  16.000000          encourage  16.000000   8.0000   8.0000
128   Default  18.000000     responsibility  18.000000   7.0000   7.0000
11    Default  20.000000            diabete  20.000000   6.0000   6.0000
82    Default  12.000000               need  12.000000   5.0000   5.0000
8     Default  23.000000               lead  23.000000   4.0000   4.0000
61    Default  12.000000            problem  12.000000   3.0000   3.0000
119   Default   9.000000               life   9.000000   2.0000   2.0000
101   Default   8.000000             choose   8.000000   1.0000   1.0000
...       ...        ...                ...        ...      ...      ...
82     Topic5  12.039260               need  12.812798   1.7785  -3.1350
86     Topic5   9.620278          available  10.397215   1.7631  -3.3593
80     Topic5   7.417521               help   8.191755   1.7415  -3.6193
109    Topic5   4.406302             affect   5.188288   1.6774  -4.1401
138    Topic5   4.273202        environment   5.045476   1.6746  -4.1708
136    Topic5   3.950233              place   4.723175   1.6621  -4.2494
21     Topic5   3.436511              prove   4.222338   1.6348  -4.3887
15     Topic5   3.120415           exercise   3.899718   1.6178  -4.4852
124    Topic5   3.080223               time   3.855911   1.6162  -4.4982
112    Topic5   2.902339             parent   3.675822   1.6045  -4.5577
128    Topic5  14.124115     responsibility  18.313564   1.5810  -2.9753
94     Topic5   2.379353           physical   3.177748   1.5514  -4.7563
139    Topic5   2.217158              spend   2.988486   1.5422  -4.8270
141    Topic5   2.211108             living   2.989042   1.5393  -4.8297
132    Topic5   2.197562               hard   2.992938   1.5319  -4.8358
111    Topic5   2.033720          education   2.809179   1.5177  -4.9133
10